In [1]:
import os, sys, shutil
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
from utils import *
from autoencoder import AutoEncoder
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import random
import keyboard
# os.chdir("..")

/export/raid1/home/kneel027/miniconda3/envs/SS/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
experiment_path = "/home/naxos2-raid25/kneel027/home/kneel027/SS_release_test/Second-Sight/experiments/pilot_preference/"
stimuli_path = "/home/naxos2-raid25/kneel027/home/kneel027/SS_release_test/Second-Sight/experiments/stimuli/"
image_path = "/home/naxos2-raid25/kneel027/home/kneel027/SS_release_test/Second-Sight/output/mindeye_extension_v6/"
subjects = [1]
idx = [i for i in range(500)]

In [4]:
for subj in subjects:
    for i in tqdm(idx):
        source = f"{experiment_path}subject{subj}/{i}/"
        dest = f"{stimuli_path}"
        os.makedirs(dest, exist_ok=True)
        shutil.copy(source + "mindeye.png", dest + f"{i}_subject{subj}_mindeye.png")
        shutil.copy(source + "ground_truth.png", dest + f"{i}_subject{subj}_ground_truth.png")
        shutil.copy(source + "secondsight.png", dest + f"{i}_subject{subj}_secondsight.png")
        
        

100%|██████████| 500/500 [00:00<00:00, 805.13it/s]


In [34]:
df_exp1 = pd.DataFrame(columns=['ground_truth', 'stim1', 'stim2', "sample", "subject", "ss_on_left", "iter_count"])

i=0
for subj in [1]: #1,2,5,7
    for sample in range(500):
        order = random.randrange(2)
        sample_names = [f"{sample}_subject{subj}_mindeye", f"{sample}_subject{subj}_secondsight"]
        
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        
        #count how many iterations the sample searched for
        iter_files = os.listdir(f"../output/mindeye_extension_v6/subject{subj}/{sample}/")
        iter_count = sum(1 for file in iter_files if "iter_" in file and file.endswith(".png"))
        
        df_exp1.loc[i] = [f"{sample}_subject{subj}_ground_truth", left_sample, right_sample, sample, subj, order == 1, iter_count]
        i+=1
print(df_exp1)

                  ground_truth                     stim1  \
0      0_subject1_ground_truth    0_subject1_secondsight   
1      1_subject1_ground_truth    1_subject1_secondsight   
2      2_subject1_ground_truth        2_subject1_mindeye   
3      3_subject1_ground_truth        3_subject1_mindeye   
4      4_subject1_ground_truth    4_subject1_secondsight   
..                         ...                       ...   
495  495_subject1_ground_truth  495_subject1_secondsight   
496  496_subject1_ground_truth  496_subject1_secondsight   
497  497_subject1_ground_truth  497_subject1_secondsight   
498  498_subject1_ground_truth  498_subject1_secondsight   
499  499_subject1_ground_truth      499_subject1_mindeye   

                        stim2  sample  subject  ss_on_left  iter_count  
0          0_subject1_mindeye       0        1        True           3  
1          1_subject1_mindeye       1        1        True           3  
2      2_subject1_secondsight       2        1       False  

In [41]:
df_exp1_tsv = df_exp1[['ground_truth', 'stim1', 'stim2']].copy().sample(frac=1)
pID_count = 0
pIDs = []
for i in range(len(df_exp1_tsv)):
    pIDs.append(i // 10)
df_exp1_tsv.insert(0, "pID", pIDs)
df_exp1_tsv.to_csv("../experiments/stimuli_order_exp1_500_short.tsv", sep="\t", index=False, header=False) 

In [37]:
df_exp2 = pd.DataFrame(columns=['ground_truth', 'stim1', 'stim2', "sample", "subject", "ss_on_left", "iter_count"])

i=0
sample_count = 500
shuffled_idx = [i for i in range(sample_count)]
random.shuffle(shuffled_idx)
for subj in [1]: #1,2,5,7
    for sample in range(sample_count):
        order = random.randrange(2)
        random_sample = shuffled_idx[sample]
        sample_names = [f"{random_sample}_subject{subj}_secondsight", f"{sample}_subject{subj}_secondsight"]
        
        left_sample = sample_names.pop(order)
        right_sample = sample_names.pop()
        
        #count how many iterations the sample searched for
        iter_files = os.listdir(f"../output/mindeye_extension_v6/subject{subj}/{sample}/")
        iter_count = sum(1 for file in iter_files if "iter_" in file and file.endswith(".png"))
        
        df_exp2.loc[i] = [f"{sample}_subject{subj}_ground_truth", left_sample, right_sample, sample, subj, order == 1, iter_count]
        i+=1
print(df_exp2)

                  ground_truth                     stim1  \
0      0_subject1_ground_truth    0_subject1_secondsight   
1      1_subject1_ground_truth  375_subject1_secondsight   
2      2_subject1_ground_truth  101_subject1_secondsight   
3      3_subject1_ground_truth    3_subject1_secondsight   
4      4_subject1_ground_truth    4_subject1_secondsight   
..                         ...                       ...   
495  495_subject1_ground_truth  367_subject1_secondsight   
496  496_subject1_ground_truth  492_subject1_secondsight   
497  497_subject1_ground_truth  495_subject1_secondsight   
498  498_subject1_ground_truth  308_subject1_secondsight   
499  499_subject1_ground_truth  499_subject1_secondsight   

                        stim2  sample  subject  ss_on_left  iter_count  
0    107_subject1_secondsight       0        1        True           3  
1      1_subject1_secondsight       1        1       False           3  
2      2_subject1_secondsight       2        1       False  

In [40]:
df_exp2_tsv = df_exp2[['ground_truth', 'stim1', 'stim2']].copy().sample(frac=1)
pID_count = 0
pIDs = []
for i in range(len(df_exp1_tsv)):
    pIDs.append(i // 10)
df_exp2_tsv.insert(0, "pID", pIDs)
df_exp2_tsv.to_csv("../experiments/stimuli_order_exp2_500_short.tsv", sep="\t", index=False, header=False) 